In [1]:
import dash
from dash import html
from dash import dcc
import numpy as np
import pandas as pd
import os
from pprint import pprint

import plotly
import plotly.graph_objs as go

In [2]:
columns = ["Bit Size", "Time Index", "Rig Activity", "Hole Depth Actual", "Delta Time"]
data = pd.read_csv('Timelog.csv', names=columns)
data.head(10)

,Bit Size,Time Index,Rig Activity,Hole Depth Actual,Delta Time
0,13.5,2018-09-28 00:00:24.0000000,Stat_Off_Bottom,23.1,24
1,13.5,2018-09-28 00:00:25.0000000,Stat_Off_Bottom,23.1,25
2,13.5,2018-09-28 00:00:26.0000000,Stat_Off_Bottom,23.1,26
3,13.5,2018-09-28 00:00:27.0000000,Stat_Off_Bottom,23.1,27
4,13.5,2018-09-28 00:00:28.0000000,Stat_Off_Bottom,23.1,28
5,13.5,2018-09-28 00:00:29.0000000,Stat_Off_Bottom,23.1,29
6,13.5,2018-09-28 00:00:30.0000000,Stat_Off_Bottom,23.1,30
7,13.5,2018-09-28 00:00:23.0000000,Stat_Off_Bottom,23.1,23
8,13.5,2018-09-28 00:00:22.0000000,Stat_Off_Bottom,23.1,22
9,13.5,2018-09-28 00:00:21.0000000,Stat_Off_Bottom,23.1,21


In [3]:
data

,Bit Size,Time Index,Rig Activity,Hole Depth Actual,Delta Time
0,13.5,2018-09-28 00:00:24.0000000,Stat_Off_Bottom,23.1,24
1,13.5,2018-09-28 00:00:25.0000000,Stat_Off_Bottom,23.1,25
2,13.5,2018-09-28 00:00:26.0000000,Stat_Off_Bottom,23.1,26
3,13.5,2018-09-28 00:00:27.0000000,Stat_Off_Bottom,23.1,27
4,13.5,2018-09-28 00:00:28.0000000,Stat_Off_Bottom,23.1,28
...,...,...,...,...,...
184499,13.5,2018-09-30 05:59:56.0000000,In-slip,2305.0,184499
184500,13.5,2018-09-30 05:59:57.0000000,In-slip,2305.0,184500
184501,13.5,2018-09-30 05:59:58.0000000,In-slip,2305.0,184501
184502,13.5,2018-09-30 05:59:59.0000000,In-slip,2305.0,184502


In [4]:
data.loc[8,'Rig Activity'] = 'In-slip'
data.loc[9,'Rig Activity'] = 'In-slip'

In [5]:
data.head(10)

,Bit Size,Time Index,Rig Activity,Hole Depth Actual,Delta Time
0,13.5,2018-09-28 00:00:24.0000000,Stat_Off_Bottom,23.1,24
1,13.5,2018-09-28 00:00:25.0000000,Stat_Off_Bottom,23.1,25
2,13.5,2018-09-28 00:00:26.0000000,Stat_Off_Bottom,23.1,26
3,13.5,2018-09-28 00:00:27.0000000,Stat_Off_Bottom,23.1,27
4,13.5,2018-09-28 00:00:28.0000000,Stat_Off_Bottom,23.1,28
5,13.5,2018-09-28 00:00:29.0000000,Stat_Off_Bottom,23.1,29
6,13.5,2018-09-28 00:00:30.0000000,Stat_Off_Bottom,23.1,30
7,13.5,2018-09-28 00:00:23.0000000,Stat_Off_Bottom,23.1,23
8,13.5,2018-09-28 00:00:22.0000000,In-slip,23.1,22
9,13.5,2018-09-28 00:00:21.0000000,In-slip,23.1,21


In [6]:
def mk_title_subtitle(title, subtitle=None):
    result = None
    _sep = '<br>'
    title = f'<b>{title}</b>'
    if not subtitle:
        result = title
    else:
        subtitle = f'{subtitle}'
        result = _sep.join((title, subtitle))
    return result

In [7]:
class Titles:
    time_vs_depth = mk_title_subtitle(
        "Time vs Hole Depth", "Actual and Plan Hole depth curves by time"
    )
    time_distribution_by_bit = mk_title_subtitle(
        "Operation Duration by Bit Size", "Actual, Plan and NPT hours by bit size"
    )
    time_distribution_by_op = mk_title_subtitle(
        "Operation Duration by Rigstate", "Actual operation hours by rigstate"
    )

In [8]:
def generate_wo_timedepth(
    df,
    samples_numbers=1920,
    figure_json=False,
    subtitle=Titles.time_vs_depth,
):

    if df is None or df.empty:
        exit
    else:
        df_plan_data = df.dropna()
        df["Hole Depth Planned"] = df["Hole Depth Actual"]
        df["Well Plan Bit Size"] = df["Bit Size"]

        def mk_hover_text(df, *args):
            text = ""
            sep = "<br>"
            for column in args:
                tag = f"{column}: "
                if pd.api.types.is_numeric_dtype(df[column].dtype):
                    series = df[column].astype(str)
                else:
                    series = df[column].astype(str)
                text = text + tag + series + sep
            return text

        df["Delta Days"] = df["Delta Time"] / 86400
        df["Time Index"] = pd.to_datetime(df["Time Index"])
        fig = go.Figure()
        df_plan_data["Delta Days"] = df_plan_data["Delta Time"] / 86400
        df_plan_data["Hole Depth Planned"] = df_plan_data["Hole Depth Actual"]
        df_plan_data["Well Plan Bit Size"] = df_plan_data["Bit Size"]
        well_plan_hover_text = mk_hover_text(
            df_plan_data,
            "Time Index",
            "Rig Activity",
            "Hole Depth Actual",
            "Hole Depth Planned",
            "Bit Size",
            "Well Plan Bit Size",
            "Delta Days",
        )
        fig_scatter_plan = go.Scatter(
            x=df_plan_data["Time Index"],
            y=df_plan_data["Hole Depth Planned"],
            text=well_plan_hover_text,
            hovertemplate="%{text}<extra></extra>",
            name="Planned Well",
        )
        fig.add_trace(fig_scatter_plan)

        def mk_scatter(df, x, y, text, name):
            fig = None
            fig = go.Scatter(
                x=df[x],
                y=df[y],
                mode="markers",
                text=text,
                hovertemplate="%{text}<extra></extra>",
                name=name,
            )
            return fig

        for activity in sorted(df["Rig Activity"].unique()):
            if activity is None:
                pass
            else:
                mask = df["Rig Activity"].apply(lambda act: act == activity)
                sub_df = df[mask]
                text = mk_hover_text(
                    sub_df,
                    "Time Index",
                    "Rig Activity",
                    "Hole Depth Actual",
                    "Hole Depth Planned",
                    "Bit Size",
                    "Well Plan Bit Size",
                    "Delta Days",
                )
                scatter_go = mk_scatter(
                    sub_df,
                    x="Time Index",
                    y="Hole Depth Actual",
                    text=text,
                    name=activity,
                )
                fig.add_trace(scatter_go)

        layout = go.Layout(
            xaxis={
                "zeroline": False,
                "zerolinewidth": 2,
                "title": "Time",
                "showgrid": True,
            },
            yaxis={
                "autorange": "reversed",
                "zeroline": False,
                "zerolinewidth": 2,
                "showgrid": True,
                "tickformat": '",d"',
                "title": "Hole Depth",
            },
            legend_x=1,
            legend_y=1,
            dragmode="select",
            hovermode="closest",
            hoverlabel=dict(font_family="Open Sans"),
            legend=dict(
                orientation="v",
                font={"family": "Open Sans"},
                x=0,
                y=1.15,
            ),
            title={
                "text": subtitle,
            },
            titlefont={"size": 15},
            font=dict(),
        )
        fig["layout"] = layout
        fig.update_layout(yaxis_tickformat=",d")
        if figure_json:
            fig = plotly.io.to_json(fig)
        return fig

In [9]:
df_2 = data.head(10)

In [10]:
figure = generate_wo_timedepth(df_2)
figure = figure.to_dict()

C:\Users\ALISHA~1.DES\AppData\Local\Temp/ipykernel_13200/1585667136.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Hole Depth Planned"] = df["Hole Depth Actual"]
C:\Users\ALISHA~1.DES\AppData\Local\Temp/ipykernel_13200/1585667136.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Well Plan Bit Size"] = df["Bit Size"]
C:\Users\ALISHA~1.DES\AppData\Local\Temp/ipykernel_13200/1585667136.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [11]:
pprint(figure)

{'data': [{'hovertemplate': '%{text}<extra></extra>',
           'name': 'Planned Well',
           'text': array(['Time Index: 2018-09-28 00:00:24.0000000<br>Rig Activity: Stat_Off_Bottom<br>Hole Depth Actual: 23.1<br>Hole Depth Planned: 23.1<br>Bit Size: 13.5<br>Well Plan Bit Size: 13.5<br>Delta Days: 0.0002777777777777778<br>',
       'Time Index: 2018-09-28 00:00:25.0000000<br>Rig Activity: Stat_Off_Bottom<br>Hole Depth Actual: 23.1<br>Hole Depth Planned: 23.1<br>Bit Size: 13.5<br>Well Plan Bit Size: 13.5<br>Delta Days: 0.00028935185185185184<br>',
       'Time Index: 2018-09-28 00:00:26.0000000<br>Rig Activity: Stat_Off_Bottom<br>Hole Depth Actual: 23.1<br>Hole Depth Planned: 23.1<br>Bit Size: 13.5<br>Well Plan Bit Size: 13.5<br>Delta Days: 0.00030092592592592595<br>',
       'Time Index: 2018-09-28 00:00:27.0000000<br>Rig Activity: Stat_Off_Bottom<br>Hole Depth Actual: 23.1<br>Hole Depth Planned: 23.1<br>Bit Size: 13.5<br>Well Plan Bit Size: 13.5<br>Delta Days: 0.0003125<br>',
  

In [ ]:
def generate_wo_timedepth(
    df
):
    
    dictionary1 = dict()
    dictionary2 = dict()

    if df is None or df.empty:
        exit
    else:
        df_plan_data = df.dropna()
        df["Hole Depth Planned"] = df["Hole Depth Actual"]
        df["Well Plan Bit Size"] = df["Bit Size"]

        def mk_hover_text(df, *args):
            text = ""
            sep = "<br>"
            for column in args:
                tag = f"{column}: "
                if pd.api.types.is_numeric_dtype(df[column].dtype):
                    series = df[column].astype(str)
                else:
                    series = df[column].astype(str)
                text = text + tag + series + sep
            return text

        df["Delta Days"] = df["Delta Time"] / 86400
        df["Time Index"] = pd.to_datetime(df["Time Index"])

        df_plan_data["Delta Days"] = df_plan_data["Delta Time"] / 86400
        df_plan_data["Hole Depth Planned"] = df_plan_data["Hole Depth Actual"]
        df_plan_data["Well Plan Bit Size"] = df_plan_data["Bit Size"]
        well_plan_hover_text = mk_hover_text(
            df_plan_data,
            "Time Index",
            "Rig Activity",
            "Hole Depth Actual",
            "Hole Depth Planned",
            "Bit Size",
            "Well Plan Bit Size",
            "Delta Days",
        )
        x1=df_plan_data["Time Index"],
        y1=df_plan_data["Hole Depth Planned"],
        hovertemplate="%{text}<extra></extra>",


        for activity in sorted(df["Rig Activity"].unique()):
            if activity is None:
                pass
            else:
                mask = df["Rig Activity"].apply(lambda act: act == activity)
                sub_df = df[mask]
                text = mk_hover_text(
                    sub_df,
                    "Time Index",
                    "Rig Activity",
                    "Hole Depth Actual",
                    "Hole Depth Planned",
                    "Bit Size",
                    "Well Plan Bit Size",
                    "Delta Days",
                )
                x2=sub_df["Time Index"],
                y2=sub_df["Hole Depth Actual"],
                dictionary2["text"] = text
                dictionary2["x"] = x2
                dictionary2["y"] = y2
                    
        dictionary1["text"] = well_plan_hover_text
        dictionary1["x"] = x1
        dictionary1["y"] = y1

        return [dictionary1, dictionary2]

In [ ]:
df_3 = data.head(1)
df_3

In [ ]:
a = generate_wo_timedepth(df_3)

In [ ]:
pprint(a)

In [ ]:
def generate_wo_timedepth_2(
    df,
    samples_numbers=1920,
    figure_json=False,
    subtitle=Titles.time_vs_depth,
):
    dictionary1 = dict()
    dictionary2 = dict()

    if df is None or df.empty:
        exit
    else:
        def mk_hover_text(df, *args):
            text = ""
            sep = "<br>"
            for column in args:
                tag = f"{column}: "
                if pd.api.types.is_numeric_dtype(df[column].dtype):
                    series = df[column].apply(get_thousand_separated_number_str)
                else:
                    series = df[column].astype(str)
                text = text + tag + series + sep
            return text

        df = prep_df_sampled(df, samples_numbers)

        df[qu.delta_days] = df[qu.delta_time] / 86400
        df[qu.time_index] = pd.to_datetime(df[qu.time_index])
        fig = go.Figure()
        df_plan_data[qu.delta_days] = df_plan_data[qu.delta_time] / 86400
        well_plan_hover_text = mk_hover_text(
            df_plan_data,
            qu.time_index,
            qu.rig_activity,
            qu.hole_depth_actual,
            qu.hole_depth_planned,
            qu.bit_size,
            qu.wp_bit_size,
            qu.delta_days,
        )
        x1=df_plan_data[qu.time_index]
        y1=df_plan_data[qu.hole_depth_planned]


        for activity in sorted(df[qu.rig_activity].unique()):
            if activity is None:
                pass
            else:
                mask = df[qu.rig_activity].apply(lambda act: act == activity)
                sub_df = df[mask]
                text = mk_hover_text(
                    sub_df,
                    qu.time_index,
                    qu.rig_activity,
                    qu.hole_depth_actual,
                    qu.hole_depth_planned,
                    qu.bit_size,
                    qu.wp_bit_size,
                    qu.delta_days,
                )
                x2=qu.time_index
                y2=qu.hole_depth_actual
                
                dictionary2["text"] = text
                dictionary2["x"] = x2
                dictionary2["y"] = y2
                
        dictionary1["text"] = well_plan_hover_text
        dictionary1["x"] = x1
        dictionary1["y"] = y1       

        return [dictionary1, dictionary2]
